#### We need to install the ktrain library. Its a light weight wrapper for keras to help train neural networks. With only a few lines of code it allows you to build models, estimate optimal learning rate, loading and preprocessing text and image data from various sources and much more. More about our approach can be found at [this](https://towardsdatascience.com/bert-text-classification-in-3-lines-of-code-using-keras-264db7e7a358) article.

In [1]:
!pip3 install ktrain==0.2.2

     |████████████████████████████████| 51kB 1.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.2.2-cp36-none-any.whl size=50418 sha256=30ac3da49ca22b1ec10c57ae08d3bab9756222409cd3463f85ef190fe66dc8d0
  Stored in directory: /root/.cache/pip/wheels/fa/21/6f/e0452e1564ccab53339b601dbd1f37ed81e34868826200e8f8
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=180438bed7964563b41411bcd633f83794ceb0ac2817898291612600cfb39f3a
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=4f32a5e7c46dec4f5dbeb040523f6c94b46048bb5eab2f85a4ee1c71b9b71206
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=c6c8607ddbc95fc5c3c2412f94d41d2d6780d3

In [2]:
#Importing
import ktrain
from ktrain import text

Using TensorFlow backend.


In [3]:
#obtain the dataset
import tensorflow as tf
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 4s 0us/step


In [4]:
%cd /root/.keras/datasets/aclImdb
!ls

/root/.keras/datasets/aclImdb
imdbEr.txt  imdb.vocab	README	test  train


In [5]:
# set path to dataset
import os.path
dataset = '/root/.keras/datasets/aclImdb'
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')
print(IMDB_DATADIR)

/root/.keras/datasets/aclImdb


## STEP 1: Preprocessing
####The texts_from_folder function will load the training and validation data from the specified folder and automatically preprocess it according to BERT's requirements. In doing so, the BERT model and vocabulary will be automatically downloaded.

In [6]:


(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(IMDB_DATADIR, 
                                                                       maxlen=500, 
                                                                       preprocess_mode='bert',
                                                                       train_test_names=['train', 
                                                                                         'test'],
                                                                       classes=['pos', 'neg'])

downloading pretrained BERT model and vocabulary...
[██████████████████████████████████████████████████]
extracting pretrained BERT model and vocabulary...
done.

cleanup downloaded zip...
done.

preprocessing train...


preprocessing test...


### STEP 2: Loading a pre trained BERT and wrapping it in a ktrain.learner object

In [7]:

model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 500




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









done.


### STEP 3: Training and Tuning the model's parameters

In [8]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 4557s 182ms/step - loss: 0.2551 - acc: 0.8944 - val_loss: 0.1694 - val_acc: 0.9342
Epoch 2/4
25000/25000 [==============================] - 4545s 182ms/step - loss: 0.1607 - acc: 0.9419 - val_loss: 0.1692 - val_acc: 0.9362
Epoch 3/4
25000/25000 [==============================] - 4545s 182ms/step - loss: 0.0875 - acc: 0.9706 - val_loss: 0.1860 - val_acc: 0.9378
Epoch 4/4
25000/25000 [==============================] - 4543s 182ms/step - loss: 0.0242 - acc: 0.9932 - val_loss: 0.2394 - val_acc: 0.9385
